# Building Cython code

### Cython code must be compiled before running it.
### Requirement: Cython, Python and a C compiler.
### Two stages

#### Translation from python to C
* A **.pyx** file is compiled by Cython to a **.c** file.

#### Compilation by C compiler
* The **.c** file is compiled by a C compiler to a **.so** file (or a **.pyd** file on Windows) or a binary executable 

## Command line building

In [1]:
# Change the working directory to helloworld/
%cd -q helloworld
!make clean

### Share library

In [2]:
# Make it helloworld.c
!cython -a helloworld.pyx

In [3]:
# Build as a shared library (.so)
!gcc -shared -pthread -fPIC -fwrapv -O3 -Wall -fno-strict-aliasing \
 -o helloworld.so helloworld.c `python2-config --cflags --ldflags`

In [4]:
# Test run
!python -c 'import helloworld'
# Clean the intermediate files
!make clean

Hello World


### Binary executable file with **--embed**

In [5]:
# Make it helloworld.c
!cython --embed -a helloworld.pyx

In [6]:
# Build as a binary executable file
!gcc -O3 -march=native -o helloworld helloworld.c `python2-config --cflags --ldflags`

In [7]:
# Test run
!./helloworld
# Clean intermediate files
!make clean

Hello World


### Embedding Cython

Typically Cython is used to create extension modules for use from Python programs. It is, however, possible to write a standalone programs in Cython. This is done via embedding the Python interpreter with the **--embed** option.

Diff of **without --embed** and **with --embed**:
```bash
+int
+main(int argc, char **argv)
+{
+    if (!argc) {
+        return __Pyx_main(0, NULL);
+    }
+    else {
+        wchar_t **argv_copy = (wchar_t **)malloc(sizeof(wchar_t*)*argc);
+        wchar_t **argv_copy2 = (wchar_t **)malloc(sizeof(wchar_t*)*argc);
+        int i, res;
+        char *oldloc;
+        if (!argv_copy || !argv_copy2) {
+            fprintf(stderr, "out of memory\\n");
+            return 1;
+        }
+        oldloc = strdup(setlocale(LC_ALL, NULL));
+        setlocale(LC_ALL, "");
+        for (i = 0; i < argc; i++) {
+            argv_copy2[i] = argv_copy[i] = __Pyx_char2wchar(argv[i]);
+            if (!argv_copy[i])
+                return 1;
+        }
+        setlocale(LC_ALL, oldloc);
+        free(oldloc);
+        res = __Pyx_main(argc, argv_copy);
+        for (i = 0; i < argc; i++) {
+            free(argv_copy2[i]);
+        }
+        free(argv_copy);
+        free(argv_copy2);
+        return res;
+    }
+}
```

## Compiling with pyximport
Aim for simple (trivial) cython build, which means
* no extra C/C++ libraries and
* no special building setup steps

In [8]:
# Compile helloworld.pyx and import via pyximport
import pyximport; pyximport.install()
import helloworld

Hello World


It can also build **.py** files

In [9]:
!rm -f helloworld2.py; echo 'print "Hello World"' > helloworld2.py

In [10]:
import pyximport; pyximport.install(pyimport=True)
import helloworld2

Hello World


## Compiling with distutils
Build with setup.py.
[setup.py](/edit/helloworld/setup.py)

```python
from distutils.core import setup
from Cython.Build import cythonize

setup(
    ext_modules = cythonize("helloworld.pyx")
)
```

In [13]:
# build extension --inplace to build the .so "in place"
!python setup.py -q build_ext --inplace -f; ls helloworld.so

helloworld.so


To rename the created library, we can use Extension()
```python
from distutils.core import setup
from Cython.Build import cythonize
from distutils.extension import Extension

extensions = [Extension('helloworld3', ['helloworld.pyx', ])]

setup(
    ext_modules = cythonize(extensions)
)
```

### Extension
For specific compile/link options, libraries to link, we can build up multiple Extension() instance manually.
```python
from distutils.core import setup
from distutils.extension import Extension
from Cython.Build import cythonize

extensions = [
    Extension("primes", ["primes.pyx"],
        include_dirs = [...],
        libraries = [...],
        library_dirs = [...]),
    # Everything but primes.pyx is included here.
    Extension("*", ["*.pyx"],
        include_dirs = [...],
        libraries = [...],
        library_dirs = [...]),
]
setup(
    name = "Finding primes",
    ext_modules = cythonize(extensions),
)
```